In [84]:
from datetime import date
from math import pi, sqrt
from openpyxl.utils import column_index_from_string
import xlsxwriter as xl #to write in template
import openpyxl
from itertools import combinations_with_replacement

In [92]:
class DesignCalculation:

    def __init__ (self, document_code,  client_name, document_name, equipment_tag, project_no, sub_project, phase, project_name, sub_project_name, jacobs_document_nomber, client_document_no, revision):
        self.__client_name = client_name
        self.__document_name = document_name
        self.__equipment_tag = equipment_tag
        self.__project_no = project_no
        self.__sub_project = sub_project
        self.__phase = phase
        self.__project_name = project_name
        self.__sub_project_name = sub_project_name
        self.__jacobs_document_nomber = jacobs_document_nomber
        self.__client_document_name = client_document_no
        self.__revision = revision
        self.__date = date.today().strftime("%d/%m/%Y")
        # eventually more specific than the Deliverable class 
        self.__code = document_code
        self.__current_level = 0
        # initialize the excel file
        self.__codification_workbook = xl.Workbook(document_name + "_codification.xlsx")
        self.__codification_worksheet = self.__codification_workbook.add_worksheet("codification")
        self.__deliverable_workbook = openpyxl.load_workbook("deliverable_template.xlsx")
        self.__deliverable_cover_page_sheet = self.__deliverable_workbook["Cover Page"]
        self.__deliverable_input_sheet = self.__deliverable_workbook["Input"]
        self.__deliverable_shell_calculation_sheet = self.__deliverable_workbook["Shell Calculation"]
        # initialize the list that will contain all the elements
        self.__all = []
    
    def generate_entry(self, name = "-", cell = "-", sheet = "-", _type = "-", symb = "-",  val = "-", unit = "-", src = "-", dpd = []):
        result = {
            "Code" : self.__code,
            "Name" : name,
            "Sheet" : sheet,
            "Cell" : cell,
            "Symbol" : symb,
            "Type" : _type,
            "Value" : val,
            "Unit" : unit,
            "Source code" : src,
            "Dependencies" : dpd
        }
        self.__all.append(result)
        self.increment_code()
        return result

    def add_level(self):
        if self.__current_level == 0:
            self.__current_level += 1
            self.__code += ".1"
        else:
            holder = self.__code.split(".")
            holder[-1] = str(int(holder[-1]) - 1)
            self.__code = ".".join(holder)
            self.__current_level += 1
            self.__code += ".1"

    def return_to_level(self, destination_level):
        remove = self.__current_level - destination_level
        self.__current_level = destination_level
        holder = self.__code.split(".")
        del holder[-remove:]
        holder[-1] = str(int(holder[-1]) + 1)
        self.__code = ".".join(holder)

    def increment_code(self):
        holder = self.__code.split(".")
        holder[-1] = str(int(holder[-1]) + 1)
        self.__code = ".".join(holder)
    
    def get_code(self):
        return self.__code
    
    def get_current_level(self):
        return self.__current_level

    def calculate(self):
        self.add_level()
        self.cover_page = self.generate_entry("Cover Page")
        self.add_level()
        self.date = self.generate_entry("Date issued", sheet = "Cover Page", cell = "B1", val = self.__date)
        self.client_name = self.generate_entry("Client name", sheet = "Cover Page", cell = "B6", val = self.__client_name)
        self.document_name = self.generate_entry("Document name", sheet = "Cover Page", cell = "B17", val = self.__document_name)
        self.equipment_tag = self.generate_entry("Equipment Tag", sheet = "Cover Page", cell = "B18", val = self.__equipment_tag)
        self.project_no = self.generate_entry("Project no", sheet = "Cover Page", cell = "E24", val = self.__project_no)
        self.sub_project = self.generate_entry("Sub Project", sheet = "Cover Page", cell = "E25", val = self.__sub_project)
        self.phase = self.generate_entry("Phase", sheet = "Cover Page", cell = "E26", val = self.__phase)
        self.project_name = self.generate_entry("Project name", sheet = "Cover Page", cell = "E27", val = self.__project_name)
        self.sub_project_no = self.generate_entry("Sub-Project name", sheet = "Cover Page", cell = "E28", val = self.__sub_project_name)
        self.jesa_document_no = self.generate_entry("Jesa document no", sheet = "Cover Page", cell = "E29", val = self.__jacobs_document_nomber)
        self.client_document_no = self.generate_entry("Client Document no", sheet = "Cover Page", cell = "E30", val = self.__client_document_name)
        self.revision = self.generate_entry("Revision", sheet = "Cover Page", cell = "E31", val = self.__revision)
        self.return_to_level(1)
        self.input = self.generate_entry("Input")
        self.add_level()

        #Input
        self.equipment_no = self.generate_entry("Equipment no.", sheet = "Input", cell = "F10", _type = "str", val = "")
        self.design_code = self.generate_entry("Design code", _type = "str", val = "API 650 12th Edition Addendum 2016")
        self.type_of_roof = self.generate_entry("Type of Roof", sheet = "Input", cell = "F14", _type = "str", val = "Open")
        self.support_type = self.generate_entry("Support Type", sheet = "Input", cell = "F15", _type = "float", val = "Self-supporting")
        self.internal_design_pressure = self.generate_entry("Internal Design Pressure", sheet = "Input", cell = "F27" ,_type = "float", symb = "Pi", val = 0, unit = "kPa", src = "PD3.1.2.3")
        self.external_design_pressure = self.generate_entry("External Design Pressure", sheet = "Input", cell = "F29" ,_type = "float", symb = "Pe", val = 0, unit = "kPa")
        self.design_temperature = self.generate_entry("Design Temperature",  sheet = "Input", cell = "F31" ,_type = "float", symb = "T", val = 50, unit = "Celsius", src = "PD3.1.2.5")
        self.product_stored = self.generate_entry("Product Stored", sheet = "Input", cell = "F32" ,_type = "str", val = "28% Phos. Acid", src = "PD3.1.2.1")
        self.specific_gravity = self.generate_entry("Specific Gravity", sheet = "Input", cell = "F33" ,_type = "float", symb = "G", val = 1.3, src = "PD3.1.2.6")
        self.tank_inside_diameter = self.generate_entry("Tank Inside Diameter", sheet = "Input", cell = "F34" , _type = "float", symb = "D", val = 15, unit = "m", src = "PD3.1.3.1")
        self.tank_height = self.generate_entry("Tank Height", sheet = "Input", cell = "F35" , _type = "float", symb = "HT", val = 11.3, unit = "m", src = "PD3.1.3.1")
        self.used_sheets = self.generate_entry("Used sheets", _type = "float", val = [2500, 2500, 2500, 2000, 1800])
        self.available_thickness =self.generate_entry("available thickness", val = [3, 5, 8, 10, 13, 15, 17, 18, 20])
        self.internal_pressure_head = self.generate_entry("Internal Pressure Head", sheet = "Input", cell = "F36" , _type = "float", symb = "HP", val = (0 if (self.internal_design_pressure["Value"] == 0 or self.internal_design_pressure["Value"] >18) else self.internal_design_pressure / (9.8 * self.specific_gravity) ), unit = "m", dpd = [self.internal_design_pressure["Code"], self.specific_gravity["Code"]])
        self.max_design_liquid_level = self.generate_entry("Max. Design Liquid Level", sheet = "Input", cell = "F37" , _type = "float", symb = "HL", val = 11.3, unit = "m", src = "API650.5.4", dpd = ["PD3.1.3.1"])
        self.max_liquid_level = self.generate_entry("Max. Liquid Level",sheet = "Input", cell = "F38" , _type = "float", symb = "HL", val = 11.3, unit = "m", src = "PD3.1.3.1")
        self.high_high_liquid_level = self.generate_entry("High High Liquid Level",sheet = "Input", cell = "F39" , _type = "float", symb = "HHLL", val = 10.7, unit = "m", src = "PD3.1.8")
        self.design_liquid_level_for_calculation = self.generate_entry("Design liquid level for calculcation",sheet = "Input", cell = "F40" , _type = "float", symb = "H", val = 11.3, unit = "m", src = "API650.5.4", dpd = ["PD3.1.3.1"])
        self.nominal_capacity = self.generate_entry("Nominal Capacity",sheet = "Input", cell = "F41" , _type = "float", val = 1996, unit = "m^3", src = "PD3.1.3.2")
        self.corrosion_allowance_of_roof = self.generate_entry("Corrosion Allowance of Roof",sheet = "Input", cell = "F42" , _type = "float", symb = "CAr", val = 0, unit = "mm", src = "PD3.1.6.3")
        self.corrosion_allowance_of_shell = self.generate_entry("Corrosion Allowance of Shell",sheet = "Input", cell = "F43" , _type = "float", symb = "CAs", val = 3, unit = "mm", src = "PD3.1.5.2")
        self.joint_efficiency = self.generate_entry("Joint Efficiency",sheet = "Input", cell = "F45" , _type = "float", val = 0.85, src = "API650.A.3.4")
        self.material_of_construction = self.generate_entry("Material of Construction",sheet = "Input", cell = "F46" , _type = "str", val = "A516Gr70", src = "PD3.1.6")
        self.material_yield_stress_at_ambiant_temperature= self.generate_entry("Material Yield Stress at Ambiant Tempreature",sheet = "Input", cell = "F47" , _type = "float", symb = "YS", val = 260, unit = "MPa", src = "API650.5.2a")
        self.material_yield_stress_at_design_temperature = self.generate_entry("Material Yield Stress at Design Temperature",sheet = "Input", cell = "F48" , _type = "float", symb = "YS*", val = 260, unit = "MPa", src = "API650.5.2a")
        self.tensile_strength_at_ambiant_temperature = self.generate_entry("Tensile Strength at Ambiant Temperature",sheet = "Input", cell = "F49" , _type = "float", symb = "TS", val = 485, unit = "MPa", src = "API650.5.2a")
        self.tensile_strength_at_design_temperature = self.generate_entry("Tensile Strength at Design Temperature",sheet = "Input", cell = "F50" , _type = "float", symb = "TS*", val = 485, unit = "MPa", src = "API650.5.2a")
        self.modulus_of_elasticity = self.generate_entry("Modulus of Elasticity",sheet = "Input", cell = "F51" , _type = "float", symb = "E", val = 199000, unit = "MPa", src = "API650.5.2a")
        self.allowable_stress_for_design_condition = self.generate_entry("Allowable Stress for Design Condition",sheet = "Input", cell = "F52" , _type = "float", symb = "Sd", val = self.calculate_allowable_stress_for_design_condition(self.material_yield_stress_at_design_temperature["Value"], self.tensile_strength_at_design_temperature["Value"]), unit = "MPa", src = "API650.5.6.2.1", dpd = [self.material_yield_stress_at_design_temperature["Code"], self.tensile_strength_at_design_temperature["Code"]])
        self.allowable_stress_for_test_condition = self.generate_entry("Allowable Stress for Test Condition",sheet = "Input", cell = "F53" , _type = "float", symb = "St", val = self.calculate_allowable_stress_for_test_condition(self.material_yield_stress_at_design_temperature["Value"], self.tensile_strength_at_design_temperature["Value"]), unit = "MPa", src = "API650.5.6.2.2", dpd = [self.material_yield_stress_at_design_temperature["Code"], self.tensile_strength_at_design_temperature["Code"]])
        self.insulation_thickness_for_shell = self.generate_entry("Insulation Thickness for Shell",sheet = "Input", cell = "F54" , _type = "float", val = 0, unit = "mm", src = "PD3.1.6.15")
        self.insulation_density_for_shell = self.generate_entry("Insulation Density for shell",sheet = "Input", cell = "F55" , _type = "float", val = 0, unit = "kg/mm^3", src = "PD3.1.6.15")
        self.insulation_thickness_for_roof = self.generate_entry("Insulation Thickness for Roof",sheet = "Input", cell = "F56" , _type = "float", val = 0, unit = "mm", src = "PD3.1.6.15")
        self.insulation_density_for_shell = self.generate_entry("Insulation Density for Roof",sheet = "Input", cell = "F57" , _type = "float", val = 0, unit = "kg/mm^3", src = "PD3.1.6.15")
        self.basic_wind_speed = self.generate_entry("Basic Wind Speed",sheet = "Input", cell = "F58" , _type = "float", symb = "V", val = "50", unit = "m/s")
        self.steel_material_density = self.generate_entry("Steel Material Density",sheet = "Input", cell = "F59" , _type = "float", symb = "ρ", val = 7850, unit = "kg/m^3")
        self.density_factor_of_water = self.generate_entry("Density Factor of Water",sheet = "Input", cell = "F60" , _type = "float", symb = "Y", val = 0.00981, unit = "MPa/m")
        self.gravitational_field_strength = self.generate_entry("Gravitational Field Strength",sheet = "Input", cell = "F61" , _type = "float", symb = "g", val = 9.81, unit = "N/Kg")
        self.brick_lining_thickness = self.generate_entry("Brick Lining Thickness",sheet = "Input", cell = "F62" , _type = "float", val = 63, unit = "mm")
        self.bricks_extended_upto = self.generate_entry("Bricks extended upto",sheet = "Input", cell = "F63" , _type = "float", val = 1000, unit = "mm")
        self.rubber_lining_thickness = self.generate_entry("Rubber Lining Thickness",sheet = "Input", cell = "F64" , _type = "float", val = 6, unit ="mm")
        self.brick_density = self.generate_entry("Brick Density",sheet = "Input", cell = "F65" , _type = "float", symb = "ρ_brick", val = 1500, unit = "kg/m^3")
        self.rubber_lining_density = self.generate_entry("Rubber Lining Density",sheet = "Input", cell = "F66" , _type = "float", symb = "ρ_rubber", val = 1400, unit = "kg/m^3")
        self.lead_density = self.generate_entry("Lead Density",sheet = "Input", cell = "F67" , _type = "float", symb = "ρ_lead", val = 11340, unit = "kg/m^3")
        #shell thickness, plate curb angle, output data
        self.return_to_level(1)
        self.shell_thk_roof_plate_curb_angle_output_data = self.generate_entry("Shell Thickness, Roof plate & Curb angle output data")
        self.is_annexf_applicable = self.generate_entry("Is Annex F applicable", _type = "bool", val = False, src = "API650.Figure.F.1")
        self.shell_thk_calculation = self.generate_entry("Shell thickness calculation")
        self.min_required_shell_thickness = self.generate_entry("min shell thickness 5.6.1.1", sheet = "Shell Calculation", cell = "K31", val = self.calculate_minimum_plate_thickness_as_per_5611())
        #1 foot method calculation
        self.add_level()
        self.one_foot_method = self.generate_entry("One foot method")
        self.design_shell_thickness_1_foot_method()
        self.L = self.generate_entry("L", sheet = "Shell Calculation", cell = "C25", _type = "float", symb = "L", val = sqrt(500 * self.tank_inside_diameter["Value"]*self.one_foot_method_table[0][7]))
        self.LH = self.generate_entry("L/H", sheet = "Shell Calculation", cell = "C26", _type = "float", symb = "L/H", val = self.L["Value"] / self.design_liquid_level_for_calculation["Value"])
        self.LHTestResult = self.generate_entry("L/HtestResult", sheet = "Shell Calculation", cell = "G26", val = "<" if self.LH["Value"] < 1000/6 else ">")
        self.variable_design_use = self.generate_entry("Variable Design-Point-Method may be used", sheet = "Shell Calculation", cell = "B27")       
        self.return_to_level(1)

    def calculate_minimum_plate_thickness_as_per_5611(self):
        if self.tank_inside_diameter["Value"] < 15:
            return 5
        elif self.tank_inside_diameter["Value"] >= 15 and self.tank_inside_diameter["Value"] < 36:
            return 6
        elif self.tank_inside_diameter["Value"] >= 36 and self.tank_inside_diameter["Value"] <= 60:
            return 8
        elif self.tank_inside_diameter["Value"] > 60:
            return 10

    def calculate_allowable_stress_for_design_condition (self, material_yield_stress_at_design_temperature, tensile_strength_at_design_temperature):
        return min(2/3 * material_yield_stress_at_design_temperature, 2/5 * tensile_strength_at_design_temperature) 

    def calculate_allowable_stress_for_test_condition (self, material_yield_stress_at_design_temperature, tensile_strength_at_design_temperature):
        return min(3/4 * material_yield_stress_at_design_temperature, 3/7 * tensile_strength_at_design_temperature)

    def design_shell_thickness_1_foot_method(self):
        table = []
        for i in range(len(self.used_sheets["Value"])):
            holder = []
            minus = 0
            for j in range(i):
                minus += self.used_sheets["Value"][j]/1000
            holder.append(self.used_sheets["Value"][i])
            holder.append(self.allowable_stress_for_design_condition["Value"])
            holder.append(self.allowable_stress_for_test_condition["Value"])
            holder.append(4.9 * (self.design_liquid_level_for_calculation["Value"] - minus - 0.3) * self.tank_inside_diameter["Value"] / self.allowable_stress_for_test_condition["Value"])
            holder.append(4.9 * self.specific_gravity["Value"]*(self.design_liquid_level_for_calculation["Value"] - minus - 0.3) * self.tank_inside_diameter["Value"] / self.allowable_stress_for_design_condition["Value"] + self.corrosion_allowance_of_shell["Value"])
            greater = max(holder[3], holder[4])
            holder.append(greater)
            if greater <= self.calculate_minimum_plate_thickness_as_per_5611():
                holder.append(self.calculate_minimum_plate_thickness_as_per_5611())
            else:
                for k in sorted(self.available_thickness["Value"]):
                    if greater <= k:
                        holder.append(k)
                        break 
            holder.append(greater - self.corrosion_allowance_of_shell["Value"])
            table.append(holder)
        self.one_foot_method_table = table
        return table

    def populate_excel(self):
        # fix first line not printing
        # for count, val in enumerate(list(self.__all[0].keys())):
        #   self.__worksheet.write(0, count, val)
        for i, value in enumerate(self.__all):
            for j, key in enumerate(value.keys()):
                self.__codification_worksheet.write(i, j, str(self.__all[i][key]))
        workbook = openpyxl.load_workbook("deliverable_template.xlsx")
        for entry in self.__all:
            if entry["Sheet"] != "-":
                sheet_name = entry["Sheet"]
                cell = entry["Cell"]
                val = entry["Value"]
                sheet = workbook[sheet_name]
                sheet[cell] = val
        # Write one-foot method table
        sheet = workbook["Shell Calculation"]
        col_letter, start_row = "D", 34
        start_column = column_index_from_string(col_letter)
        
        for i, row_data in enumerate(self.one_foot_method_table):
            for j, value in enumerate(row_data):
                cell_row = start_row + i
                cell_column = start_column + j
                sheet.cell(row=cell_row, column=cell_column, value=value)
        
        # Save the result
        workbook.save("result_deliverable.xlsx")
        self.__codification_workbook.close()


In [93]:
#document_code,  client_name, document_name, equipment_tag, project_no, sub_project, phase, project_name, sub_project_name, jacobs_document_nomber, client_document_no, revision
proto = DesignCalculation("MVC1.1", "OCP", "28 % Phosphoric Acid Storage Tank ", "318AAR02", "Q3510", "OSBL", "IV", "DAP Units 107D, 107E & 107F", "OSBL", "JEC-Q3510-OSBL-IV/FEG.05a/00016", "313A-FEG.05a-00016", "A")
proto.calculate()
proto.populate_excel()